# Operace "Blackout"
> Tvým úkolem je napsat skript, který zanalyzuje seznam dostupných souborů a vybere přesně ty, které ti přinesou **největší celkovou hodnotu** (kredity), a přitom se stihnou stáhnout v časovém limitu.

### Parametry:
- 300s
- Max rychlost 20MB/s

In [ ]:
total_time_to_download = 300
max_speed = 20

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

with open("data.txt", "r") as f:
    line_data = f.read().splitlines()
    for line in line_data:
        try:
            # Vytáhneme si data z řádku
            target = line.split("|")
            name = target[0].strip()
            size = int(target[1].strip()[6:-2])
            credit_value = int(target[2].strip()[5:-8])
            # Poměr
            ratio = credit_value / size
            #print(time_to_download)
            # Přidání "normalizovaných" dat do dataframe
            df = pd.concat([df, pd.DataFrame([{"name": name, "size": size,"time_to_download": size / max_speed, "credit_value": credit_value, "ratio": ratio}])], ignore_index=True)
        except Exception as e:
            print(e)

df.sort_values(by="ratio", ascending=False, inplace=True) # Srovnání podle poměru kredity/čas

#### První dataframe
- Obsahuje normalizované hodnoty ze souboru
- Přidány sloupce pro čas stahování a poměr kredity/čas
- Pomocí poměru vybíráme soubory, které nám jsou přínosem

In [ ]:
df

## Smyčka a výsledný dataframe
- Vybíráme soubory / řádky dokud nám zbyde čas od shora srovnaného DFka
- Kontrolujeme, jestli se nám ještě vejde do zbývajícího času, pokud ne, tak tento řádek zahodíme a pokračujeme dál
- Pokud se nám vejde, přidáme ho do výsledného dataframe a odečtememe čas stahování tohoto souboru od zbývajícího času
- Odebereme již zpracovaný řádek z origo DF, abychom ho už nebrali v úvahu při dalším výběru
- 

In [ ]:
result_df = pd.DataFrame() # Výsledný dataframe, který bude obsahovat vybrané soubory
while total_time_to_download > 0:
    if df.empty:
        print("No more files to select.")
        break # Pokud už nejsou žádné soubory k výběru, ukončíme smyčku

    # Ze srovnaného DF vybereme řádek s nejvyšším poměrem
    max_ratio_row = df.loc[df['ratio'].idxmax()]

    # Zkontrolujeme, jestli čas stahování tohoto souboru nepřesahuje zbývající čas, jinak vyhodíme tento řádek a pokračujeme dál
    if max_ratio_row['time_to_download'] > total_time_to_download:
        print("FIle too big, skipping: ", max_ratio_row['name'])
        df = df.drop(max_ratio_row.name)
        continue

    # Přidáme do výsledného DF
    result_df = pd.concat([result_df, max_ratio_row.to_frame().T], ignore_index=True)
    # Odečteme čas stahování tohoto souboru od zbývajícího času
    total_time_to_download -= max_ratio_row['time_to_download']
    # Odebereme již zpracovaný řádek z origo DF
    df = df.drop(max_ratio_row.name)



#### Výsledný dataframe
- Obsahuje pouze vybrané soubory, které nám přinesou největší hodnotu a stihnou se stáhnout v časovém limitu

In [ ]:
result_df

In [ ]:
print("Totální počet kreditů:", result_df["credit_value"].sum())
print("Celkový čas stahování:", result_df["time_to_download"].sum())
print("Zbývající čas:", total_time_to_download)